In [1]:
import pickle

with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

In [9]:
import numpy as np

def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds + 1e-8) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.random.choice(len(preds), p=preds)

def generate_text(model, tokenizer, seq_len, seed_text, n_words=25, temperature=1.0):
    result = []
    input_text = seed_text

    for _ in range(n_words):
        encoded = tokenizer.texts_to_sequences([input_text])[0]
        encoded = encoded[-seq_len:]
        padded = np.pad(encoded, (seq_len - len(encoded), 0), mode='constant')
        padded = padded.reshape(1, -1)

        preds = model.predict(padded, verbose=0)[0]
        pred_id = sample(preds, temperature)

        next_word = tokenizer.index_word.get(pred_id, "<UNK>")
        result.append(next_word)
        input_text += ' ' + next_word

    return ' '.join([seed_text] + result)

In [10]:
from keras.models import load_model

lstm_model = load_model("best_model_lstm.h5")
gru_model = load_model("best_model_gru.h5")

In [11]:
seed_text = "to be or not to"
seq_len = 50

print("LSTM Output:")
print(generate_text(lstm_model, tokenizer, seq_len=50, seed_text="to be or not to", n_words=25, temperature=0.8))


print("\nGRU Output:")
print(generate_text(gru_model, tokenizer, seq_len=50, seed_text="to be or not to", n_words=25, temperature=0.8))

LSTM Output:
to be or not to give it stir with paid hither both nay he in grace as up us yet keep they'll be not the faith i can ha sight

GRU Output:
to be or not to not with nay good lands and i will never say not on a enemy of them not with very humphrey with your of they empress


In [12]:
# Load your original text (used during training)
with open("alllines.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Convert to tokens
tokens = tokenizer.texts_to_sequences([text])[0]

In [13]:
import random

def get_random_seed_text(tokenizer, tokens, seq_len=50):
    start_idx = random.randint(0, len(tokens) - seq_len - 1)
    seed_ids = tokens[start_idx : start_idx + seq_len]
    seed_words = [tokenizer.index_word.get(i, "<UNK>") for i in seed_ids]
    return ' '.join(seed_words)

In [14]:
seed_text = get_random_seed_text(tokenizer, tokens, seq_len=50)

print("Random Seed:")
print(seed_text)

print("\nLSTM Output:")
print(generate_text(lstm_model, tokenizer, seq_len=50, seed_text=seed_text, n_words=25, temperature=1.0))

print("\nGRU Output:")
print(generate_text(gru_model, tokenizer, seq_len=50, seed_text=seed_text, n_words=25, temperature=1.0))


Random Seed:
do mock thee thus stamp rave and fret that i may sing and dance thou wouldst be fee'd i see to make me sport york cannot speak unless he wear a crown a crown for york and lords bow low to him hold you his hands whilst i do set

LSTM Output:
do mock thee thus stamp rave and fret that i may sing and dance thou wouldst be fee'd i see to make me sport york cannot speak unless he wear a crown a crown for york and lords bow low to him hold you his hands whilst i do set so bashful what song down thus from thy lusts lies bring her marry and they base it eyes in you i'll succession in grossly as

GRU Output:
do mock thee thus stamp rave and fret that i may sing and dance thou wouldst be fee'd i see to make me sport york cannot speak unless he wear a crown a crown for york and lords bow low to him hold you his hands whilst i do set dismiss him memory here cheer make away the third as the warlike despair oil and her poor fight my other ranks comforted to france tis
